In [40]:
!pip install ultralytics -q

In [8]:
!pip install deep-sort-realtime -q

In [42]:
# Disable warnings in the notebook to maintain clean output cells
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml
from PIL import Image
from ultralytics import YOLO
from IPython.display import Video
from deep_sort_realtime.deepsort_tracker import DeepSort

In [88]:
# Fungsi untuk melacak dan menghitung kendaraan berdasarkan jenisnya
def track_and_count_vehicles(frame, detections, tracker, totalCount, class_name, gates, limits):
    # Gunakan DeepSORT untuk melacak objek berdasarkan deteksi YOLOv8
    tracks = tracker.update_tracks(detections, frame=frame)
    
    # List untuk menyimpan bounding box
    bboxes = []
    
    # Loop untuk menggambar bounding boxes dan ID objek pada frame
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()  # bounding box format (left, top, right, bottom)
        x1, y1, x2, y2 = map(int, ltrb)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2

        # Simpan bounding box dan track_id untuk visualisasi di luar
        bboxes.append((x1, y1, x2, y2, track_id))
        
        # Update deteksi kendaraan yang melewati gerbang
        for i, gate in enumerate(gates, start=1):
            # Cek apakah kendaraan berada dalam batas gerbang
            if gate[0] < cx < gate[2] and gate[1] - 15 < cy < gate[1] + 15:
                # Jika kendaraan belum ada dalam daftar, tambahkan
                if totalCount.count(track_id) == 0:
                    totalCount.append(track_id)
                    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (255, 0, 255), 5)

    # Kembalikan bounding box dan totalCount
    return bboxes, totalCount

In [108]:
import cv2
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Inisialisasi YOLOv8 model
model = YOLO("yolov8x.pt")

# Video input
video_path = "/kaggle/working/sample_video.mp4"
cap = cv2.VideoCapture(video_path)

# Dapatkan dimensi video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Definisikan VideoWriter untuk menyimpan video dalam format .avi
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('result_video.avi', fourcc, 30, (width, height))

# Membaca class names dari file
with open('/kaggle/input/classes/classes.txt', 'r') as f:
    classNames = f.read().splitlines()

# Definisikan batas gerbang
limits = [17, 151, 568, 284]

# Variabel untuk menyimpan total kendaraan yang terdeteksi
totalCount_car = []
totalCount_bus = []
totalCount_truck = []

# Definisikan posisi gerbang
gates = [[14, 157, 74, 174],
    [74, 174, 138, 190],
    [138, 190, 203, 203],
    [203, 203, 271, 218],
    [271, 218, 339, 232],
    [411, 248, 485, 264],
    [485, 264, 561, 284]]
 # Gunakan DeepSORT untuk melacak objek berdasarkan deteksi YOLOv8
tracker_car = DeepSort(max_age=30, n_init=1, max_iou_distance=0.9)
tracker_bus = DeepSort(max_age=30, n_init=1, max_iou_distance=0.9)
tracker_truck = DeepSort(max_age=30, n_init=1, max_iou_distance=0.9)

# Loop utama untuk membaca frame dan memprosesnya
frame_num = 0
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame_num += 1
    
    # Deteksi objek menggunakan YOLOv8
    results = model(frame, conf=0.8)

    # Ekstraksi bounding boxes dan skor dari hasil YOLOv8
    detections_car = []
    detections_bus = []
    detections_truck = []
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())
            currentClasses = classNames[class_id]
            if currentClasses == 'car':
                detections_car.append(([x1, y1, (x2-x1), (y2-y1)], conf, class_id))
            if currentClasses == 'bus':
                detections_bus.append(([x1, y1, (x2-x1), (y2-y1)], conf, class_id))
            if currentClasses == 'truck':
                detections_truck.append(([x1, y1, (x2-x1), (y2-y1)], conf, class_id))
    
    # Gambar garis batas pada frame
    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
        
        # Hitung mobil
    bboxes_car, totalCount_car = track_and_count_vehicles(frame, detections_car, tracker_car, totalCount_car, 'car', gates, limits)
    print(bboxes_car)
    # Hitung bus
    bboxes_bus, totalCount_bus = track_and_count_vehicles(frame, detections_bus, tracker_bus, totalCount_bus, 'bus', gates, limits)

    # Hitung truk
    bboxes_truck, totalCount_truck = track_and_count_vehicles(frame, detections_truck, tracker_truck, totalCount_truck, 'truck', gates, limits)

    # Visualisasi bounding box dan informasi di luar fungsi
    for (x1, y1, x2, y2, track_id) in bboxes_car:
        cv2.circle(frame, ((x1 + x2) // 2, (y1 + y2) // 2), 5, (255, 0, 255), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id} car', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    for (x1, y1, x2, y2, track_id) in bboxes_bus:
        cv2.circle(frame, ((x1 + x2) // 2, (y1 + y2) // 2), 5, (255, 0, 255), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f'ID: {track_id} bus', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    for (x1, y1, x2, y2, track_id) in bboxes_truck:
        cv2.circle(frame, ((x1 + x2) // 2, (y1 + y2) // 2), 5, (255, 0, 255), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f'ID: {track_id} truck', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
    
    totalVehicles = len(totalCount_car)+len(totalCount_bus)+len(totalCount_truck)
    # Tampilkan jumlah kendaraan yang telah dihitung
    cv2.putText(frame, f'Total Vehicles: {totalVehicles}', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(frame, f'Car: {len(totalCount_car)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(frame, f'Bus: {len(totalCount_bus)}', (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(frame, f'Truck: {len(totalCount_truck)}', (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Simpan frame ke video output
    out.write(frame)

# Tutup video capture dan jendela OpenCV
cap.release()
out.release()


0: 384x640 1 bus, 62.9ms
Speed: 2.1ms preprocess, 62.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[]

0: 384x640 1 bus, 61.6ms
Speed: 2.7ms preprocess, 61.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[]
Detected bus: ID=1, BBox=(225, 111, 343, 173)

0: 384x640 1 bus, 34.4ms
Speed: 2.3ms preprocess, 34.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[]
Detected bus: ID=1, BBox=(225, 111, 343, 173)

0: 384x640 1 bus, 34.5ms
Speed: 1.9ms preprocess, 34.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[]
Detected bus: ID=1, BBox=(225, 111, 343, 173)

0: 384x640 1 bus, 34.3ms
Speed: 1.9ms preprocess, 34.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[]
Detected bus: ID=1, BBox=(225, 111, 343, 173)

0: 384x640 1 bus, 30.1ms
Speed: 1.8ms preprocess, 30.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[]
Detected bus: ID=1, BBox=(225, 111, 343, 173)

0: 384x640 1 

In [109]:
# Convert the .avi video generated by the YOLOv8 prediction to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i result_video.avi result_video.mp4
Video('result_video.mp4', embed=True, width=960)

In [110]:
!python --version

Python 3.10.13
